In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import copy,time
from sklearn.preprocessing import LabelEncoder

In [2]:
f_train = os.path.expanduser('~/Desktop/titanic_train.csv')
f_test = os.path.expanduser('~/Desktop/titanic_test.csv')

In [5]:
df_train = pd.read_csv(f_train)
df_test = pd.read_csv(f_test)

train_y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId','Survived'], axis=1)
df_test = df_test.drop(['PassengerId'], axis=1)
df = pd.concat([df_train,df_test])

In [8]:
obj_f = []
for dtype,feature in zip(df.dtypes,df.columns):
    if dtype == 'object':
        obj_f.append(feature)
print(f'{len(obj_f)} \nobject features:{obj_f}\n')

df = df[obj_f]
df = df.fillna('None')
train_num = train_y.shape[0]

5 
object features:['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']



In [10]:
# label encoding + logistic regression
df_temp = pd.DataFrame()
for c in df.columns:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
train_X = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_y, cv=5).mean())
df_temp.head()

0.780004837244799


/Users/yu-hsuantseng/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/yu-hsuantseng/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/yu-hsuantseng/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/yu-hsuantseng/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/yu-hsuantseng/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: 

,Name,Sex,Ticket,Cabin,Embarked
0,155,1,720,185,3
1,286,0,816,106,0
2,523,0,914,185,3
3,422,0,65,70,3
4,22,1,649,185,3


In [11]:
df.select_dtypes(include=['object']).apply(pd.Series.nunique)

Name        1307
Sex            2
Ticket       929
Cabin        187
Embarked       4
dtype: int64

In [12]:
count_df = df.groupby(['Cabin'])['Name'].agg({'Cabin_count':'size'}).reset_index()
df = pd.merge(df,count_df,on=['Cabin'],how='left')
count_df.sort_values(by=['Cabin_count'],ascending=False).head(10)

/Users/yu-hsuantseng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.


,Cabin,Cabin_count
185,None,1014
80,C23 C25 C27,6
184,G6,5
47,B57 B59 B63 B66,5
60,B96 B98,4
180,F2,4
117,D,4
79,C22 C26,4
181,F33,4
183,F4,4


In [13]:
df.head()

,Name,Sex,Ticket,Cabin,Embarked,Cabin_count
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S,1014
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C,2
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S,1014
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S,2
4,"Allen, Mr. William Henry",male,373450,None,S,1014


In [14]:
df_temp = pd.DataFrame()
for c in obj_f:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
df_temp['Cabin_count'] = df['Cabin_count']
train_x = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator,train_x,train_y,cv=5).mean())

0.7856230275549181


/Users/yu-hsuantseng/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/yu-hsuantseng/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/yu-hsuantseng/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/yu-hsuantseng/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/yu-hsuantseng/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: 

In [15]:
# plus hash
df_temp = pd.DataFrame()
for c in obj_f:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
df_temp['Cabin_hash'] = df['Cabin'].map(lambda x:hash(x)%10)
train_x = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator,train_x,train_y,cv=5).mean())

0.7766591590074002


/Users/yu-hsuantseng/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/yu-hsuantseng/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/yu-hsuantseng/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/yu-hsuantseng/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/yu-hsuantseng/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: 